<a href="https://colab.research.google.com/github/Saultr21/IA-Y-BIGDATA/blob/main/PRO/CartPole/CartPole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agente de Aprendizaje por Refuerzo para MountainCarContinuous-v0

Este notebook implementa un agente de aprendizaje por refuerzo profundo (DRL) para resolver el entorno MountainCarContinuous-v0 de OpenAI Gym.

## Descripción del Problema

El entorno MountainCarContinuous-v0 consiste en un automóvil ubicado en un valle entre dos montañas. El objetivo es conducir el auto hasta la cima de la montaña a la derecha. Sin embargo, el motor del auto no es lo suficientemente potente para escalar la montaña directamente. Por lo tanto, el agente debe aprender a balancear el auto hacia adelante y hacia atrás para ganar suficiente impulso para llegar a la cima.

- **Estado**: Vector de 2 valores [posición, velocidad]
- **Acción**: Valor continuo entre -1 y 1 (fuerza aplicada)
- **Recompensa**: -0.1*acción² en cada paso + 100 si alcanza la meta

A diferencia de CartPole, este es un problema con espacio de acción continuo, por lo que requiere modificaciones significativas en el agente.

In [ ]:
# 📌 INSTALAR DEPENDENCIAS EN GOOGLE COLAB
!sudo apt-get update --fix-missing
!sudo apt-get install -y xvfb ffmpeg
!pip install -U gym
!pip install pygame
!pip install keras
!pip install tensorflow
!pip install pyvirtualdisplay
!pip install matplotlib

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,321 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,708 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/

In [ ]:
# 🎥 HABILITAR EL RENDERIZADO EN COLAB
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

print("¡Virtual Display iniciado correctamente!")

# 📌 IMPORTAR LIBRERÍAS NECESARIAS
import numpy as np
import pandas as pd
import gym
import random
import cv2
import base64
from collections import deque
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from IPython.display import HTML
import time as time_module
import os
from keras.models import Sequential, load_model
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, message=".*np.bool8.*")

# 📌 REGISTRAR MÉTRICAS PARA TENSORBOARD
tensorboard_callback = TensorBoard(log_dir="./logs")

¡Virtual Display iniciado correctamente!


## Modificación 1: Adaptación del Agente para Espacio de Acción Continuo

A continuación, modificamos la clase DQLAgent para manejar acciones continuas. Los principales cambios incluyen:

1. La arquitectura de la red neuronal ahora tiene una salida de dimensión 1 (un valor continuo)
2. La función de activación de la capa de salida es `tanh` para producir valores entre -1 y 1
3. El método `act()` ahora devuelve un valor continuo en lugar de un índice discreto
4. Se implementa un método de exploración basado en ruido gaussiano

In [ ]:
# 🔥 DEFINICIÓN DEL AGENTE PARA ESPACIO DE ACCIÓN CONTINUO
class ContinuousDQLAgent():
    def __init__(self, env, model_path=None):
        # Tamaño del estado y acción del entorno
        self.state_size = env.observation_space.shape[0]  # 2 para MountainCarContinuous
        self.action_size = env.action_space.shape[0]      # 1 para MountainCarContinuous

        self.gamma = 0.99       # Factor de descuento
        self.learning_rate = 0.001  # Tasa de aprendizaje

        # Se elimina el epsilon-greedy para usar exploración basada en ruido
        # self.epsilon = 1.0
        # self.epsilon_decay = 0.995
        # self.epsilon_min = 0.01

        self.noise_scale = 0.3   # Escala inicial del ruido
        self.noise_decay = 0.995 # Factor de decaimiento del ruido
        self.noise_scale_min = 0.01  # Escala mínima del ruido

        # Se aumenta el buffer de replay
        self.memory = deque(maxlen=20000)

        # Se elimina el contador de batch (ya no se entrena cada 20 llamadas)
        # self.batch_counter = 0

        if model_path and os.path.exists(model_path):
            print(f"Cargando modelo desde {model_path}...")
            self.model = load_model(model_path)
            print("Modelo cargado correctamente.")
        else:
            print("Creando un nuevo modelo...")
            self.model = self.build_model()

    def build_model(self):
        # Se amplía la red neuronal
        model = Sequential()
        model.add(Input(shape=(self.state_size,)))  # Usar Input en lugar de input_shape
        model.add(Dense(32, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(self.action_size, activation='tanh'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        # Se guardan TODAS las experiencias (se elimina el filtrado del 50%)
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        # Se usa la predicción de la red con ruido gaussiano para explorar (se elimina epsilon-greedy)
        action = self.model.predict(state, verbose=0)[0]
        return np.clip(action + np.random.normal(0, self.noise_scale, self.action_size), -1, 1)

    def replay(self, batch_size):
        # Entrenar en cada paso si hay suficientes muestras
        if len(self.memory) < batch_size:
            return

        # Se elimina el contador y se entrena siempre que sea posible
        minibatch = random.sample(self.memory, batch_size)

        states = np.zeros((batch_size, self.state_size))
        targets = np.zeros((batch_size, self.action_size))

        for i, (state, action, reward, next_state, done) in enumerate(minibatch):
            # Predicción actual
            target = self.model.predict(state, verbose=0)[0]

            if not done:
                target_val = reward + self.gamma * self.model.predict(next_state, verbose=0)[0][0]
            else:
                target_val = reward

            target[0] = target_val

            states[i] = state
            targets[i] = target

        # Entrenamiento con el batch completo
        self.model.fit(states, targets, epochs=1, verbose=0, batch_size=batch_size)

    # Se elimina el método adaptiveEGreedy y se utiliza el decaimiento del ruido al final de cada episodio.
    def decay_noise(self):
        if self.noise_scale > self.noise_scale_min:
            self.noise_scale *= self.noise_decay

## Modificación 2: Entorno y Entrenamiento

Ahora modificamos el código de inicialización del entorno y entrenamiento:

1. Cambiamos `CartPole-v1` a `MountainCarContinuous-v0`
2. Ajustamos el número de episodios y los parámetros de entrenamiento
3. Implementamos un seguimiento de las recompensas para visualizar el progreso del entrenamiento

In [ ]:
# 📌 INICIALIZAR EL ENTORNO
env = gym.make('MountainCarContinuous-v0', render_mode="rgb_array")

# 📌 ENTRENAMIENTO DEL AGENTE - ACTUALIZACIÓN POR EPISODIO
if __name__ == "__main__":
    model_path = 'mountaincar_dql.keras'
    episode_file = 'current_episode.txt'

    # Cargar el episodio actual desde el archivo, o comenzar desde el episodio 0
    if os.path.exists(episode_file):
        with open(episode_file, 'r') as f:
            start_episode = int(f.read())
    else:
        start_episode = 0

    agent = ContinuousDQLAgent(env, model_path=model_path)
    batch_size = 32   # Batch size aumentado
    total_episodes = 50  # Se aumenta el número de episodios para un entrenamiento más robusto

    rewards_per_episode = []
    max_positions = []

    start_time = time_module.time()
    last_update_time = start_time

    print("Iniciando entrenamiento del agente...")
    print("Este proceso puede tardar varios minutos. Se mostrarán actualizaciones periódicas.")

    for e in range(start_episode, total_episodes):
        episode_start_time = time_module.time()

        state = env.reset()
        if isinstance(state, tuple):
            state = state[0]
        state = np.reshape(state, [1, 2])

        total_reward = 0
        time_step = 0
        max_position = -1.2

        print(f"Iniciando episodio {e+1}/{total_episodes}...")

        while True:
            current_time = time_module.time()
            if current_time - last_update_time > 3:  # Actualización cada 3 segundos
                print(f"Episodio {e+1}/{total_episodes}, paso {time_step}, pos={max_position:.3f} - Procesando...", flush=True)
                last_update_time = current_time

            # Obtener acción con exploración por ruido
            action = agent.act(state)
            action_reshaped = action.reshape(1)

            # Ejecutar acción en el entorno
            next_state, reward, terminated, truncated, _ = env.step(action_reshaped)
            done = terminated or truncated
            next_state = np.reshape(next_state, [1, 2])

            current_position = next_state[0][0]
            max_position = max(max_position, current_position)

            # Se usa la recompensa original; se puede añadir shaping si se desea
            modified_reward = reward

            # Guardar experiencia
            agent.remember(state, action, modified_reward, next_state, done)

            # Entrenar en cada paso (con mayor frecuencia)
            agent.replay(batch_size)

            state = next_state
            total_reward += reward
            time_step += 1

            # Visualización del progreso cada 5 pasos
            if time_step % 5 == 0:
                elapsed = time_module.time() - episode_start_time
                print(f"Paso {time_step}, Pos: {max_position:.3f}, Tiempo: {elapsed:.1f}s")

            if done or time_step >= 100:
                episode_duration = time_module.time() - episode_start_time
                print(f'\nEpisodio {e+1} completado en {episode_duration:.1f}s:')
                print(f'Pasos: {time_step}, Reward: {total_reward:.2f}, Max Pos: {max_position:.4f}')

                rewards_per_episode.append(total_reward)
                max_positions.append(max_position)
                break

        # Al finalizar el episodio, reducir el ruido de exploración
        agent.decay_noise()

        # Guardar el modelo y actualizar el archivo de episodio
        agent.model.save(model_path)
        with open(episode_file, 'w') as f:
            f.write(str(e + 1))

        clear_output(wait=True)

        # Estadísticas y gráficas del progreso
        print(f"\n--- Resumen de progreso ---")
        print(f"Progreso: {e+1}/{total_episodes} episodios ({(e+1)/total_episodes*100:.1f}%)")
        print(f"Mejor posición: {max(max_positions):.4f} (meta: 0.5)")
        print(f"Tiempo transcurrido: {(time_module.time() - start_time)/60:.1f} min")

        plt.figure(figsize=(10, 4))
        plt.plot(max_positions, 'r-')
        plt.axhline(y=0.5, color='g', linestyle='--')
        plt.title(f'Progreso del Entrenamiento (Episodio {e+1}/{total_episodes})')
        plt.ylabel('Posición Máxima')
        plt.xlabel('Episodio')
        plt.grid(True)
        plt.show()

        plt.figure(figsize=(10, 4))
        plt.plot(rewards_per_episode, 'b-')
        plt.title(f'Recompensa por Episodio (Episodio {e+1}/{total_episodes})')
        plt.ylabel('Recompensa Total')
        plt.xlabel('Episodio')
        plt.grid(True)
        plt.show()

        print("-" * 50)

    print("\nEntrenamiento finalizado!")
    if os.path.exists(episode_file):
        os.remove(episode_file)

Creando un nuevo modelo...
Iniciando entrenamiento del agente...
Este proceso puede tardar varios minutos. Se mostrarán actualizaciones periódicas.
Iniciando episodio 1/50...
Paso 5, Pos: -0.584, Tiempo: 1.6s
Paso 10, Pos: -0.554, Tiempo: 2.7s
Episodio 1/50, paso 12, pos=-0.541 - Procesando...
Paso 15, Pos: -0.519, Tiempo: 3.8s
Paso 20, Pos: -0.483, Tiempo: 4.8s
Paso 25, Pos: -0.454, Tiempo: 6.2s
Episodio 1/50, paso 25, pos=-0.454 - Procesando...
Paso 30, Pos: -0.438, Tiempo: 6.6s
Episodio 1/50, paso 32, pos=-0.438 - Procesando...
Episodio 1/50, paso 33, pos=-0.438 - Procesando...
Episodio 1/50, paso 34, pos=-0.438 - Procesando...
Paso 35, Pos: -0.438, Tiempo: 33.0s
Episodio 1/50, paso 35, pos=-0.438 - Procesando...
Episodio 1/50, paso 36, pos=-0.438 - Procesando...


KeyboardInterrupt: 

In [ ]:
# 🎥 GRABAR VIDEO DEL AGENTE
def record_video(env, agent, video_path="mountaincar_video.mp4", frames=500):
    obs = env.reset()
    if isinstance(obs, tuple):
        obs = obs[0]
    obs = np.reshape(obs, [1, 2])  # 2 dimensiones para MountainCarContinuous

    frame_shape = (600, 400)
    out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, frame_shape)

    for _ in range(frames):
        frame = env.render()

        if frame is None:
            print("⚠️ Error: El frame renderizado es None.")
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        frame = cv2.resize(frame, frame_shape)

        out.write(frame)

        action = agent.model.predict(obs, verbose=0)
        action_reshaped = np.reshape(action, [1])  # Reshape para el entorno
        obs, _, terminated, truncated, _ = env.step(action_reshaped)
        done = terminated or truncated
        obs = np.reshape(obs, [1, 2])

        if done:
            break

    out.release()
    env.close()
    print("🎥 Video guardado correctamente en", video_path)

def display_video(video_path):
    video_file = open(video_path, "rb").read()
    video_url = f"data:video/mp4;base64,{base64.b64encode(video_file).decode()}"
    return HTML(f'<video width="600" height="400" controls><source src="{video_url}" type="video/mp4"></video>')

# 📌 LLAMAR A LA FUNCIÓN PARA GRABAR Y MOSTRAR EL VIDEO
record_video(env, agent, "mountaincar_video.mp4")
display_video("mountaincar_video.mp4")

🎥 Video guardado correctamente en mountaincar_video.mp4
